## Azure AI Search - Ingesting embeddings with Cohere using the AmlSkill

In [ ]:
! pip install azure-identity azure-storage-blob python-dotenv cohere --quiet
! pip install --upgrade --pre azure-search-documents --quiet

## Load Environment Variables

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv(override=True)

# Environment Variables
AZURE_AI_STUDIO_COHERE_ENDPOINT = os.getenv("AZURE_AI_STUDIO_COHERE_ENDPOINT")
AZURE_AI_STUDIO_COHERE_KEY = os.getenv("AZURE_AI_STUDIO_COHERE_KEY")
SEARCH_SERVICE_ENDPOINT = os.getenv("SEARCH_SERVICE_ENDPOINT")
SEARCH_SERVICE_API_KEY = os.getenv("SEARCH_SERVICE_API_KEY")  
BLOB_CONNECTION_STRING = os.getenv("BLOB_CONNECTION_STRING")
BLOB_CONTAINER_NAME = "csv-files"
CSV_FILE_PATH = os.path.join("data", "csv", "AG_news_samples.csv")
INDEX_NAME = "csv-sample"


## Authentication for Azure AI Search

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential

def authenticate_azure_search(use_aad_for_search=True):
    if use_aad_for_search:
        credential = DefaultAzureCredential()
    else:
        credential = AzureKeyCredential(SEARCH_SERVICE_API_KEY)
    return credential

azure_search_credential = authenticate_azure_search(use_aad_for_search=False)


## Connect to Azure Blob Storage

In [ ]:
from azure.storage.blob import BlobServiceClient
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

def get_blob_service_client(connection_string):
    """Initialize and return the BlobServiceClient."""
    return BlobServiceClient.from_connection_string(connection_string)

def get_container_client(blob_service_client, container_name):
    """Get or create the container client."""
    container_client = blob_service_client.get_container_client(container_name)
    if not container_client.exists():
        try:
            container_client.create_container()
            logging.info(f"Created container: {container_name}")
        except Exception as e:
            logging.error(f"Error creating container: {container_name}, {e}")
    return container_client

def upload_file_to_blob(container_client, file_path, container_name):
    """Upload a file to the specified blob container."""
    file_name = os.path.basename(file_path)
    blob_client = container_client.get_blob_client(file_name)

    if not blob_client.exists():
        try:
            with open(file_path, "rb") as data:
                container_client.upload_blob(name=file_name, data=data)
            logging.info(f"Uploaded {file_name} to {container_name}")
        except Exception as e:
            logging.error(f"Error uploading {file_name} to {container_name}: {e}")
    else:
        logging.info(f"{file_name} already exists in {container_name}")

# Main workflow
blob_service_client = get_blob_service_client(BLOB_CONNECTION_STRING)
container_client = get_container_client(blob_service_client, BLOB_CONTAINER_NAME)
upload_file_to_blob(container_client, CSV_FILE_PATH, BLOB_CONTAINER_NAME)

## Create a blob data source connector on Azure AI Search

In [ ]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

def create_or_update_data_source(indexer_client, container_name, connection_string, index_name):
    """
    Create or update a data source connection for Azure Cognitive Search.
    
    Parameters:
    - indexer_client: The SearchIndexerClient instance.
    - container_name: Name of the blob container.
    - connection_string: Connection string for the blob storage.
    - index_name: Name of the index to associate with this data source.
    """
    try:
        container = SearchIndexerDataContainer(name=container_name)
        data_source_connection = SearchIndexerDataSourceConnection(
            name=f"{index_name}-blob",
            type="azureblob",
            connection_string=connection_string,
            container=container
        )
        data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)
        logging.info(f"Data source '{data_source.name}' created or updated")
    except Exception as e:
        logging.error(f"Failed to create or update data source: {e}")

# Assuming 'search_service_endpoint', 'credential', 'blob_container_name', 'blob_connection_string', and 'index_name' are defined
indexer_client = SearchIndexerClient(SEARCH_SERVICE_ENDPOINT, azure_search_credential)

# Call the function to create or update the data source
create_or_update_data_source(indexer_client, BLOB_CONTAINER_NAME, BLOB_CONNECTION_STRING, INDEX_NAME)


## Create a search index

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticSearch,
    SemanticPrioritizedFields,
    SimpleField,
    SemanticField,
    SearchIndex
)
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

def create_fields():
    """Creates the fields for the search index based on the specified schema."""
    return [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True, filterable=True),
        SearchField(name="title", type=SearchFieldDataType.String, searchable=True),
        SearchField(name="description", type=SearchFieldDataType.String, searchable=True),
        SearchField(name="label", type=SearchFieldDataType.String, facetable=True,filterable=True),
        SearchField(name="label_int", type=SearchFieldDataType.Int32, sortable=True, filterable=True, facetable=True),
        SearchField(
            name="vector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=1024,
            vector_search_profile_name="myHnswProfile",
        ),
    ]

def create_vector_search_configuration():
    """Creates the vector search configuration."""
    return VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw",
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE,
                ),
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw"
            )
        ]
    )

def create_semantic_search_configuration():
    """Creates the semantic search configuration."""
    return SemanticSearch(configurations=[
        SemanticConfiguration(
            name="my-semantic-config",
            prioritized_fields=SemanticPrioritizedFields(
                title_field=SemanticField(field_name="title"),
                content_fields=[SemanticField(field_name="description")]
            ),
        )
    ])

def create_search_index(index_name, fields, vector_search, semantic_search):
    """Creates or updates the search index."""
    index = SearchIndex(
        name=index_name, 
        fields=fields, 
        vector_search=vector_search, 
        semantic_search=semantic_search
    )
    try:
        result = index_client.create_or_update_index(index)
        logging.info(f"{result.name} created")
    except Exception as e:
        logging.error(f"Failed to create or update index: {e}")

# Assuming 'endpoint', 'credential', 'azure_openai_endpoint', 'azure_openai_embedding_deployment', and 'azure_openai_key' are defined
index_client = SearchIndexClient(endpoint=SEARCH_SERVICE_ENDPOINT, credential=azure_search_credential)
fields = create_fields()
vector_search = create_vector_search_configuration()
semantic_search = create_semantic_search_configuration()

# Create the search index with the adjusted schema
create_search_index(INDEX_NAME, fields, vector_search, semantic_search)


## Create a skillset

In [ ]:
from azure.search.documents.indexes.models import (
    AzureMachineLearningSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    SearchIndexerSkillset
)
from azure.search.documents.indexes import SearchIndexerClient
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

def create_embedding_skill(azure_ai_studio_cohere_endpoint, azure_ai_studio_cohere_key):
    """Defines the embedding skill for generating embeddings via AI Studio Cohere."""
    return AzureMachineLearningSkill(
        description="Skill to generate embeddings via AI Studio Cohere",
        context="/document",
        scoring_uri=f"{azure_ai_studio_cohere_endpoint}/v1/embed",
        authentication_key=azure_ai_studio_cohere_key,
        inputs=[
            InputFieldMappingEntry(name="texts", source="=[$(/document/description)]"),
            InputFieldMappingEntry(name="input_type", source="='search_document'"),
            InputFieldMappingEntry(name="truncate", source="='END'"), # Trim end of input if necessary
            InputFieldMappingEntry(name="embedding_types", source="=['float']"),
        ],
        outputs=[
            OutputFieldMappingEntry(name="embeddings", target_name="aml_vector_object")
        ],
    )

def create_skillset(client, skillset_name, embedding_skill):
    """Creates or updates the skillset with a an embedding skill."""
    skillset = SearchIndexerSkillset(
        name=skillset_name,
        description="Skillset for generating embeddings",
        skills=[embedding_skill],
    )
    try:
        client.create_or_update_skillset(skillset)
        logging.info(f"{skillset.name} created")
    except Exception as e:
        logging.error(f"Failed to create or update skillset {skillset_name}: {e}")

skillset_name = f"{INDEX_NAME}-skillset"
client = SearchIndexerClient(endpoint=SEARCH_SERVICE_ENDPOINT, credential=azure_search_credential)

embedding_skill = create_embedding_skill(AZURE_AI_STUDIO_COHERE_ENDPOINT, AZURE_AI_STUDIO_COHERE_KEY)

create_skillset(client, skillset_name, embedding_skill)


## Create and Run Indexer

In [ ]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import (
    SearchIndexer,
    IndexerExecutionStatus,
    FieldMapping,
    IndexingParameters,
    IndexingParametersConfiguration,
    BlobIndexerParsingMode,
)
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)


def create_and_run_indexer(
    indexer_client, indexer_name, skillset_name, index_name, data_source_name
):
    """
    Creates an indexer, applies it to a given index, and runs the indexing process.
    """
    try:
        indexer = SearchIndexer(
            name=indexer_name,
            description="Indexer to index documents and generate embeddings",
            skillset_name=skillset_name,
            target_index_name=index_name,
            data_source_name=data_source_name,
            # Indexing parameters to correctly parse CSV files
            parameters=IndexingParameters(
                batch_size=100,
                configuration=IndexingParametersConfiguration(
                    parsing_mode=BlobIndexerParsingMode.DELIMITED_TEXT,
                    first_line_contains_headers=True,
                    query_timeout=None,
                ),
            ),
            output_field_mappings=[FieldMapping(source_field_name="/document/aml_vector_object/float/0", target_field_name="vector")]
        )

        # Create or update the indexer
        indexer_client.create_or_update_indexer(indexer)
        logging.info(f"{indexer_name} created or updated.")

        # Run the indexer
        indexer_client.run_indexer(indexer_name)
        logging.info(
            f"{indexer_name} is running. If queries return no results, please wait a bit and try again."
        )
    except Exception as e:
        logging.error(f"Failed to create or run indexer {indexer_name}: {e}")


# Assuming 'endpoint', 'credential', 'index_name', 'skillset_name', and 'data_source_name' are defined correctly
data_source_name = f"{INDEX_NAME}-blob"
indexer_name = f"{INDEX_NAME}-indexer"  # Ensure `index_name` is defined and correctly references your index
indexer_client = SearchIndexerClient(
    endpoint=SEARCH_SERVICE_ENDPOINT, credential=azure_search_credential
)

# Call the function to create and run the indexer
create_and_run_indexer(
    indexer_client,
    indexer_name,
    skillset_name,
    INDEX_NAME,
    data_source_name
)

## Poll for indexer completion

In [ ]:
indexer_last_result = indexer_client.get_indexer_status(indexer_name).last_result
indexer_status = IndexerExecutionStatus.IN_PROGRESS if indexer_last_result is None  else indexer_last_result.status

while(indexer_status == IndexerExecutionStatus.IN_PROGRESS): 
    indexer_last_result = indexer_client.get_indexer_status(indexer_name).last_result
    indexer_status = IndexerExecutionStatus.IN_PROGRESS if indexer_last_result is None  else indexer_last_result.status

logging.info(f"Indexer '{indexer_name}' finished with status '{indexer_status}'.")

## Perform a Vector Search

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
import cohere

co = cohere.Client(api_key=AZURE_AI_STUDIO_COHERE_KEY, base_url=f"{AZURE_AI_STUDIO_COHERE_ENDPOINT}/v1")

def generate_embeddings(texts, input_type="search_query"):
    model = "embed-english-v3.0"
    # Ensure texts is a list
    if isinstance(texts, str):
        texts = [texts]

    response = co.embed(
        texts=texts,
        model=model,
        input_type=input_type,
        embedding_types=["float"],
    )
    return [embedding for embedding in response.embeddings.float]

# Query for vector search
query = "oil prices"

# Generate query embeddings
# Use input_type="search_query" for query embeddings
query_embeddings = generate_embeddings(query, input_type="search_query")

search_client = SearchClient(SEARCH_SERVICE_ENDPOINT, INDEX_NAME, azure_search_credential)

vector_query = VectorizedQuery(
    vector=query_embeddings[0], k_nearest_neighbors=3, fields="vector"
)

results = search_client.search(
    search_text=None,  # No search text for pure vector search
    vector_queries=[vector_query],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Description: {result['description']}")
    print(f"Label: {result['label']}")
    print(f"Score: {result['@search.score']}\n")